## Prerequisites:
* .venv/bin/pip install mysql-connector-python pymssql sqlalchemy

In [1]:
import pandas as pd
import numpy as np
import pymssql
import mysql.connector

In [2]:
sqlselect = "SELECT studentid, school, sex, age, famsize,\
    medu AS mothereducation, fedu AS fathereducation,\
    traveltime, studytime, failures, famrel, freetime,\
    goout, g1 AS gradeperiod1, g2 AS gradeperiod2,\
    g3 AS gradeperiod3 From studentmath"

server = 'pdcc.c9sqqzd5fulv.us-west-2.rds.amazonaws.com'
user = "pdccuser"
password = "pdccpass"
database = "pdcctest"
conn = pymssql.connect(server=server,
    user=user, password=password, database=database)
studentmath = pd.read_sql(sqlselect,conn)
conn.close()

/tmp/ipykernel_74725/3681628002.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  studentmath = pd.read_sql(sqlselect,conn)


Alternative method to fix warning "UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy." when connecting via pymssql + mysql connector

In [3]:
from sqlalchemy import create_engine

sqlselect = "SELECT studentid, school, sex, age, famsize,\
     medu AS mothereducation, fedu AS fathereducation,\
     traveltime, studytime, failures, famrel, freetime,\
     goout, g1 AS gradeperiod1, g2 AS gradeperiod2,\
     g3 AS gradeperiod3 From studentmath"

# Create SQLAlchemy engine
server = 'pdcc.c9sqqzd5fulv.us-west-2.rds.amazonaws.com'
user = "pdccuser"
password = "pdccpass"
database = "pdcctest"

# Create connection string for SQL Server
connection_string = f"mssql+pymssql://{user}:{password}@{server}/{database}"
engine = create_engine(connection_string)

# Read SQL query using SQLAlchemy engine
studentmath = pd.read_sql(sqlselect, engine)

# Close all connections
engine.dispose()

In [4]:
studentmath.dtypes

studentid          object
school             object
sex                object
age                 int64
famsize            object
mothereducation     int64
fathereducation     int64
traveltime          int64
studytime           int64
failures            int64
famrel              int64
freetime            int64
goout               int64
gradeperiod1        int64
gradeperiod2        int64
gradeperiod3        int64
dtype: object

In [5]:
studentmath.shape

(395, 16)

In [16]:
studentmath.head()

,gradeperiod1,gradeperiod2,gradeperiod3,school,sex,age,famsize,mothereducation,fathereducation,traveltime,studytime,freetime,failures,famrel,goout
studentid,,,,,,,,,,,,,,,
001,5,6,6,GP,F,18,GT3,4:higher ed,4:higher ed,2,2,3:neutral,0,4:good,4:high
002,5,5,6,GP,F,17,GT3,1:k-4,1:k-4,1,2,3:neutral,0,5:excellent,3:neutral
003,7,8,10,GP,F,15,LE3,1:k-4,1:k-4,1,2,3:neutral,3,4:good,2:low
004,15,14,15,GP,F,15,GT3,4:higher ed,2:5-9,1,3,2:low,0,3:neutral,2:low
005,6,10,10,GP,F,16,GT3,3:secondary ed,3:secondary ed,1,2,3:neutral,0,4:good,2:low


In [7]:
newcolorder = ['studentid', 'gradeperiod1',
    'gradeperiod2', 'gradeperiod3', 'school',
    'sex', 'age', 'famsize', 'mothereducation',
    'fathereducation', 'traveltime',
    'studytime', 'freetime', 'failures',
    'famrel', 'goout']

studentmath = studentmath[newcolorder]
studentmath.studentid.count()

np.int64(395)

In [8]:
studentmath.studentid.nunique()

395

In [9]:
studentmath.set_index('studentid', inplace=True)

In [10]:
studentmath.count()

gradeperiod1       395
gradeperiod2       395
gradeperiod3       395
school             395
sex                395
age                395
famsize            395
mothereducation    395
fathereducation    395
traveltime         395
studytime          395
freetime           395
failures           395
famrel             395
goout              395
dtype: int64

In [11]:
setvalues= \
 {"famrel":{1:"1:very bad",2:"2:bad",
    3:"3:neutral",4:"4:good",5:"5:excellent"},
 "freetime":{1:"1:very low",2:"2:low",
    3:"3:neutral",4:"4:high",5:"5:very high"},
 "goout":{1:"1:very low",2:"2:low",3:"3:neutral",
    4:"4:high",5:"5:very high"},
 "mothereducation":{0:np.nan,1:"1:k-4",2:"2:5-9",
    3:"3:secondary ed",4:"4:higher ed"},
 "fathereducation":{0:np.nan,1:"1:k-4",2:"2:5-9",
    3:"3:secondary ed",4:"4:higher ed"}}
studentmath.replace(setvalues, inplace=True)
setvalueskeys = [k for k in setvalues]

In [13]:
setvalueskeys

['famrel', 'freetime', 'goout', 'mothereducation', 'fathereducation']

In [14]:
studentmath[setvalueskeys].memory_usage(index=False)

famrel             3160
freetime           3160
goout              3160
mothereducation    3160
fathereducation    3160
dtype: int64

In [15]:
for col in studentmath[setvalueskeys].columns:
    studentmath[col] = studentmath[col].astype('category')

studentmath[setvalueskeys].memory_usage(index=False)

famrel             607
freetime           607
goout              607
mothereducation    599
fathereducation    599
dtype: int64

In [22]:
studentmath['famrel'].value_counts(sort=False, normalize=True)

famrel
1:very bad     0.020253
2:bad          0.045570
3:neutral      0.172152
4:good         0.493671
5:excellent    0.268354
Name: proportion, dtype: float64

In [23]:
studentmath[['freetime', 'goout']].apply(pd.Series.value_counts, sort=False, normalize=True)

,freetime,goout
1:very low,0.048101,0.058228
2:low,0.162025,0.260759
3:neutral,0.397468,0.329114
4:high,0.291139,0.217722
5:very high,0.101266,0.134177


In [24]:
studentmath[['mothereducation', 'fathereducation']].apply(pd.Series.value_counts, sort=False, normalize=True)

,mothereducation,fathereducation
1:k-4,0.150510,0.208651
2:5-9,0.262755,0.292621
3:secondary ed,0.252551,0.254453
4:higher ed,0.334184,0.244275


Test connecting to clickhouse database

In [6]:
import clickhouse_connect
import json

# Create client
client = clickhouse_connect.get_client(
    host='192.168.139.103',
    port=8123,  # HTTP port
    username='default',
    password='P@ssword',
    database='airbyte_internal'
)


In [7]:

# Execute query
query = "SELECT * FROM default_raw__stream_task"

In [8]:
df = client.query_df(query)

In [9]:
df.head(10)

,_airbyte_raw_id,_airbyte_data,_airbyte_extracted_at,_airbyte_loaded_at
0,00067307-36f8-4bb6-97da-55e84e235c7c,"{""id"":""86eqv0m2k"",""custom_item_id"":0,""name"":""M...",2025-05-08 01:38:46.863000+00:00,NaT
1,045942ce-89f9-4bcf-914a-c54a510dcdf1,"{""id"":""86eqvqjc7"",""custom_item_id"":0,""name"":""U...",2025-05-08 01:38:33.868000+00:00,NaT
2,0ab96a39-63b1-4f39-8f6a-02abd6cd922c,"{""id"":""86erknbah"",""custom_item_id"":0,""name"":""[...",2025-05-08 01:38:46.860000+00:00,NaT
3,10a8630b-d46c-43c7-a262-6decabdab5ad,"{""id"":""86erwywqy"",""custom_item_id"":0,""name"":""P...",2025-05-08 01:38:44.770000+00:00,NaT
4,1107d23b-2af2-4864-a1af-42dcfe3278ed,"{""id"":""86et9g2d8"",""custom_item_id"":0,""name"":""A...",2025-05-08 01:38:45.806000+00:00,NaT
5,1a33e74a-2ffc-49fc-858c-8ed1d52ee131,"{""id"":""86ermamwu"",""custom_item_id"":0,""name"":""[...",2025-05-08 01:38:39.778000+00:00,NaT
6,2208da4a-315e-4df5-a4b7-35d50be9f51b,"{""id"":""86eqwgxhv"",""custom_item_id"":0,""name"":""E...",2025-05-08 01:38:46.862000+00:00,NaT
7,226f47c5-2368-4f9a-becc-bfaa4843488c,"{""id"":""86erj1z8r"",""custom_item_id"":0,""name"":""S...",2025-05-08 01:38:35.878000+00:00,NaT
8,2299e0ed-653e-4e5d-9cc3-0d4a36baa931,"{""id"":""86erwywqv"",""custom_item_id"":0,""name"":""P...",2025-05-08 01:38:44.770000+00:00,NaT
9,2f414506-4982-4fd2-a80b-4dfeac414f20,"{""id"":""86er20ghb"",""custom_item_id"":0,""name"":""G...",2025-05-08 01:38:46.366000+00:00,NaT


In [10]:
df.shape

(63, 4)

In [11]:
df_parsed = pd.json_normalize(df['_airbyte_data'].apply(json.loads))

In [12]:
df_parsed.head()

,id,custom_item_id,name,text_content,description,orderindex,date_created,date_updated,archived,assignees,...,folder.hidden,folder.access,space.id,due_date,start_date,priority.color,priority.id,priority.orderindex,priority.priority,date_done
0,86eqv0m2k,0,Manage Public IP Addresses on Netbox,Backlog URL https://iij-vn.backlog.com/view/TU...,Backlog URL https://iij-vn.backlog.com/view/TU...,120303728.00011110000000000000000000000000,1732590275456,1741834545320,False,"[{'id': 1970400046, 'username': 'Tin Le', 'col...",...,False,True,90182833159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,86eqvqjc7,0,Upgrade Gen1.8-HCM07 vCD & others,,,120303154.00000000000000000000000000000000,1732787100220,1742202538958,False,"[{'id': 1970400046, 'username': 'Tin Le', 'col...",...,False,True,90181841801,1754082000000,1754082000000,NaN,NaN,NaN,NaN,NaN
2,86erknbah,0,[SSL cert by FreeIPA] Research how we manage F...,Backlog URL https://iij-vn.backlog.com/view/TU...,Backlog URL https://iij-vn.backlog.com/view/TU...,120305482.00024560000000000000000000000000,1740127783430,1742283834722,False,"[{'id': 1970400046, 'username': 'Tin Le', 'col...",...,False,True,90182833159,1756501200000,1729198800000,#f50000,1,1,urgent,NaN
3,86erwywqy,0,Projector,NaN,NaN,39895684.00000000000000000000000000000000,1742439748096,1742439758336,False,"[{'id': 1970400046, 'username': 'Tin Le', 'col...",...,False,True,90182833159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86et9g2d8,0,Aws Account Management process,,,120472395.00000000000000000000000000000000,1745812815052,1745812859501,False,"[{'id': 1970400046, 'username': 'Tin Le', 'col...",...,False,True,90182833159,1745874000000,1745442000000,NaN,NaN,NaN,NaN,NaN


In [13]:
df_parsed['list.name']

0                                                  List
1     [TU-project] Upgrade VMware software for whole...
2                                                  List
3                                                Assets
4     [BU-project] Migrate & Upgrade HIGIO Website P...
                            ...                        
58    [TU-project] Upgrade VMware software for whole...
59    [TU-project] Upgrade VMware software for whole...
60                                                 List
61    [TU-project] Migration of gen1 storages to Pur...
62                      [TU-project] Move SCv3000 to S3
Name: list.name, Length: 63, dtype: object

In [14]:
import requests
from dotenv import load_dotenv
import os

In [15]:
# Load environment variables
load_dotenv()

# Get API token from environment variable
api_token = os.getenv('CLICKUP_API_TOKEN')

print(api_token)

pk_1970400046_7NF5Q5LQW5GHUB94KDKKTW57CSMDB3TE


In [16]:
# Create headers for API requests
headers = {
    "Authorization": api_token,
    "Content-Type": "application/json"
}

# Example request
def get_tasks(list_id):
    """Get tasks from a specific list"""
    url = f"https://api.clickup.com/api/v2/list/{list_id}/task"
    
    params = {
        'archived': True,
        'subtasks': True,
        'include_closed': True
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        tasks = response.json()
        
        if 'tasks' in tasks:
            # Convert to DataFrame
            df = pd.DataFrame(tasks['tasks'])
            
            # # Select important columns
            # columns = [
            #     'id', 'name', 'status', 'date_created', 
            #     'date_updated', 'date_closed', 'creator'
            # ]
            
            # df = df[columns]
            
            # # Convert timestamps
            # df['date_created'] = pd.to_datetime(df['date_created'], unit='ms')
            # df['date_updated'] = pd.to_datetime(df['date_updated'], unit='ms')
            # df['date_closed'] = pd.to_datetime(df['date_closed'], unit='ms')

            # Convert timestamp columns if they exist
            timestamp_columns = ['date_created', 'date_updated', 'date_closed', 'start_date', 'due_date']
            for col in timestamp_columns:
                if col in df.columns:
                    df[col] = pd.to_datetime(df[col], unit='ms')
            
            return df
    
    return None

In [17]:
list_id = "901803362279"  # Replace with your actual list ID
tasks_df = get_tasks(list_id)

if tasks_df is not None:
    display(tasks_df)  # Jupyter's display function for better DataFrame visualization

/tmp/ipykernel_10544/3639136110.py:44: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df[col] = pd.to_datetime(df[col], unit='ms')
/tmp/ipykernel_10544/3639136110.py:44: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df[col] = pd.to_datetime(df[col], unit='ms')
/tmp/ipykernel_10544/3639136110.py:44: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without

,id,custom_id,custom_item_id,name,text_content,description,status,orderindex,date_created,date_updated,...,linked_tasks,locations,team_id,url,sharing,permission_level,list,project,folder,space
0,86erq5j7f,None,0,[Change request] Replace SSL for HI GIO 1.8 HN...,,,"{'status': 'concept', 'id': 'p90182833159_stPY...",237655484.00000000000000000000000000000000,2025-03-04 04:49:47.697,2025-03-04 04:52:00.739,...,[],[],9018530843,https://app.clickup.com/t/86erq5j7f,"{'public': False, 'public_share_expires_on': N...",create,"{'id': '901803362279', 'name': 'List', 'access...","{'id': '90184208322', 'name': 'Side Tasks', 'h...","{'id': '90184208322', 'name': 'Side Tasks', 'h...",{'id': '90182833159'}
1,86erknbwf,None,0,[Change request] [Feb-2025] Renew SSL cert for...,,,"{'status': 'Closed', 'id': 'p90182833159_h6oUt...",237655412.00000000000000000000000000000000,2025-02-21 08:52:12.500,2025-02-23 16:13:48.903,...,[],[],9018530843,https://app.clickup.com/t/86erknbwf,"{'public': False, 'public_share_expires_on': N...",create,"{'id': '901803362279', 'name': 'List', 'access...","{'id': '90184208322', 'name': 'Side Tasks', 'h...","{'id': '90184208322', 'name': 'Side Tasks', 'h...",{'id': '90182833159'}
2,86erknbwd,None,0,[Change request] [Feb-2025] Renew SSL cert for...,,,"{'status': 'Closed', 'id': 'p90182833159_h6oUt...",237655411.00000000000000000000000000000000,2025-02-21 08:52:12.140,2025-02-27 02:49:09.426,...,[],[],9018530843,https://app.clickup.com/t/86erknbwd,"{'public': False, 'public_share_expires_on': N...",create,"{'id': '901803362279', 'name': 'List', 'access...","{'id': '90184208322', 'name': 'Side Tasks', 'h...","{'id': '90184208322', 'name': 'Side Tasks', 'h...",{'id': '90182833159'}
3,86erknbwb,None,0,[Change request] [Feb-2025] Renew SSL cert for...,,,"{'status': 'Closed', 'id': 'p90182833159_h6oUt...",237655410.00000000000000000000000000000000,2025-02-21 08:52:11.766,2025-02-23 16:13:30.102,...,[],[],9018530843,https://app.clickup.com/t/86erknbwb,"{'public': False, 'public_share_expires_on': N...",create,"{'id': '901803362279', 'name': 'List', 'access...","{'id': '90184208322', 'name': 'Side Tasks', 'h...","{'id': '90184208322', 'name': 'Side Tasks', 'h...",{'id': '90182833159'}
4,86erknbwa,None,0,drgw-hcmc01a.higio.net | Cert: SSL certificate...,,,"{'status': 'Closed', 'id': 'p90182833159_h6oUt...",237655409.00000000000000000000000000000000,2025-02-21 08:52:11.377,2025-02-23 16:13:12.965,...,[],[],9018530843,https://app.clickup.com/t/86erknbwa,"{'public': False, 'public_share_expires_on': N...",create,"{'id': '901803362279', 'name': 'List', 'access...","{'id': '90184208322', 'name': 'Side Tasks', 'h...","{'id': '90184208322', 'name': 'Side Tasks', 'h...",{'id': '90182833159'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,86eqaf9nm,None,0,add deadline,,,"{'status': 'Closed', 'id': 'p90182833159_h6oUt...",39745246.00000000000000000000000000000000,2024-09-19 04:22:47.787,2025-03-05 06:23:47.294,...,[],[],9018530843,https://app.clickup.com/t/86eqaf9nm,"{'public': False, 'public_share_expires_on': N...",create,"{'id': '901803362279', 'name': 'List', 'access...","{'id': '90184208322', 'name': 'Side Tasks', 'h...","{'id': '90184208322', 'name': 'Side Tasks', 'h...",{'id': '90182833159'}
57,86eqa7tmn,None,0,TU's tasks,,,"{'status': 'Closed', 'id': 'p90182833159_h6oUt...",39745234.00000000000000000000000000000000,2024-09-18 09:57:10.151,2025-03-05 06:23:47.294,...,[],[],9018530843,https://app.clickup.com/t/86eqa7tmn,"{'public': False, 'public_share_expires_on': N...",create,"{'id': '901803362279', 'name': 'List', 'access...","{'id': '90184208322', 'name': 'Side Tasks', 'h...","{'id': '90184208322', 'name': 'Side Tasks', 'h...",{'id': '90182833159'}
58,86eqa48tf,None,0,add more expected tasks and targets [Oki & mr....,,,"{'status': 'Closed', 'id': 'p90182833159_h6oUt...",39745218.00000249600000000000000000000000,2024-09-18 03:50:31.781,2025-03-05 06:23:47.294,...,[],[],9018530843,http

In [18]:
tasks_df.isnull().sum()

id                   0
custom_id           61
custom_item_id       0
name                 0
text_content         0
description          0
status               0
orderindex           0
date_created         0
date_updated         0
date_closed         28
date_done           28
archived             0
creator              0
assignees            0
group_assignees      0
watchers             0
checklists           0
tags                 0
parent               8
top_level_parent     8
priority            56
due_date            56
start_date          59
points              61
time_estimate       61
custom_fields        0
dependencies         0
linked_tasks         0
locations            0
team_id              0
url                  0
sharing              0
permission_level     0
list                 0
project              0
folder               0
space                0
dtype: int64

In [22]:
tasks_df.shape

(61, 38)

In [25]:
# Show last 5 columns
display(tasks_df.loc[:, tasks_df.columns[-3]])

0     {'id': '90184208322', 'name': 'Side Tasks', 'h...
1     {'id': '90184208322', 'name': 'Side Tasks', 'h...
2     {'id': '90184208322', 'name': 'Side Tasks', 'h...
3     {'id': '90184208322', 'name': 'Side Tasks', 'h...
4     {'id': '90184208322', 'name': 'Side Tasks', 'h...
                            ...                        
56    {'id': '90184208322', 'name': 'Side Tasks', 'h...
57    {'id': '90184208322', 'name': 'Side Tasks', 'h...
58    {'id': '90184208322', 'name': 'Side Tasks', 'h...
59    {'id': '90184208322', 'name': 'Side Tasks', 'h...
60    {'id': '90184208322', 'name': 'Side Tasks', 'h...
Name: project, Length: 61, dtype: object